
# Завдання 3 — Hugging Face (pipeline): (a) + (b)

## (a) Класифікація (2 моделі)
- `mrm8488/bert-tiny-finetuned-enron-spam-detection`
- `mrm8488/bert-tiny-finetuned-sms-spam-detection`

Рахуємо:
- accuracy
- classification_report
- зберігаємо CSV

## (b) Українські моделі (pipeline)
Показуємо приклади:
- translation (uk→en): `Helsinki-NLP/opus-mt-uk-en`
- summarization (uk): `ukr-models/uk-summarizer`
- zero-shot (multilingual): `MoritzLaurer/mDeBERTa-v3-base-mnli-xnli`

In [9]:

from pathlib import Path
import pandas as pd
import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import pipeline

print("Torch:", torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
device_id = 0 if device == "cuda" else -1
print("Device:", device)


Torch: 2.9.1+cu128
Device: cpu


In [10]:

LAB_DIR = Path(r"/home/kali/Downloads/data_anal/lab4")
csv_path = LAB_DIR / "email_spam.csv"
assert csv_path.exists(), f"Не знайдено {csv_path}"

df = pd.read_csv(csv_path).dropna()
df["text_all"] = (df["title"].astype(str) + " " + df["text"].astype(str)).str.strip()
df["label"] = df["type"].astype(str).str.lower()

df["label"] = df["label"].replace({"ham":"not spam", "notspam":"not spam", "non-spam":"not spam", "non spam":"not spam"})
df.loc[~df["label"].isin(["spam", "not spam"]), "label"] = df["label"].apply(lambda x: "spam" if "spam" in x else "not spam")

print(df["label"].value_counts())
df.head()


label
not spam    58
spam        26
Name: count, dtype: int64


,title,text,type,text_all,label
0,?? the secrets to SUCCESS,"Hi James,\n\nHave you claim your complimentary...",spam,"?? the secrets to SUCCESS Hi James,\n\nHave yo...",spam
1,?? You Earned 500 GCLoot Points,"\nalt_text\nCongratulations, you just earned\n...",not spam,?? You Earned 500 GCLoot Points \nalt_text\nCo...,not spam
2,?? Your GitHub launch code,"Here's your GitHub launch code, @Mortyj420!\n ...",not spam,?? Your GitHub launch code Here's your GitHub ...,not spam
3,[The Virtual Reward Center] Re: ** Clarifications,"Hello,\n \nThank you for contacting the Virtua...",not spam,[The Virtual Reward Center] Re: ** Clarificati...,not spam
4,"10-1 MLB Expert Inside, Plus Everything You Ne...","Hey Prachanda Rawal,\n\nToday's newsletter is ...",spam,"10-1 MLB Expert Inside, Plus Everything You Ne...",spam


In [11]:

_, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df["label"])

X_test = test_df["text_all"].tolist()
y_true = test_df["label"].tolist()

MAX_EVAL = 500
X_test = X_test[:MAX_EVAL]
y_true = y_true[:MAX_EVAL]
len(X_test)


26

In [12]:
MODEL_A = "mrm8488/bert-tiny-finetuned-enron-spam-detection"
MODEL_B = "mrm8488/bert-tiny-finetuned-sms-spam-detection"

clf_a = pipeline("text-classification", model=MODEL_A, device=device_id)
clf_b = pipeline("text-classification", model=MODEL_B, device=device_id)

# ✅ У email_spam.csv листи можуть бути дуже довгі.
# BERT має максимум ~512 токенів, тому треба ЯВНО задати max_length, інакше буде RuntimeError (1227 vs 512).
MAX_LEN = 256   # 256 швидше на CPU; якщо хочеш — постав 512

def shorten_text(s: str, max_chars: int = 4000) -> str:
    # додатковий запобіжник: обрізаємо сирий текст, щоб не годувати модель мегабайтами
    s = str(s)
    return s[:max_chars]

def predict_pipe(clf, texts, batch_size=8):
    out = []
    for i in range(0, len(texts), batch_size):
        batch = [shorten_text(t) for t in texts[i:i+batch_size]]
        preds = clf(
            batch,
            truncation=True,
            max_length=MAX_LEN,
            padding=True,
        )
        for p in preds:
            lab = str(p["label"]).lower()
            if "spam" in lab or lab.endswith("1") or "label_1" in lab:
                out.append("spam")
            else:
                out.append("not spam")
    return out

y_pred_a = predict_pipe(clf_a, X_test)
y_pred_b = predict_pipe(clf_b, X_test)

acc_a = accuracy_score(y_true, y_pred_a)
acc_b = accuracy_score(y_true, y_pred_b)

print("Accuracy A:", acc_a)
print("Accuracy B:", acc_b)

Device set to use cpu
Device set to use cpu


Accuracy A: 0.5
Accuracy B: 0.5384615384615384


In [13]:

print("=== MODEL A report ===")
print(classification_report(y_true, y_pred_a, digits=4))

print("=== MODEL B report ===")
print(classification_report(y_true, y_pred_b, digits=4))


=== MODEL A report ===
              precision    recall  f1-score   support

    not spam     1.0000    0.2778    0.4348        18
        spam     0.3810    1.0000    0.5517         8

    accuracy                         0.5000        26
   macro avg     0.6905    0.6389    0.4933        26
weighted avg     0.8095    0.5000    0.4708        26

=== MODEL B report ===
              precision    recall  f1-score   support

    not spam     0.6667    0.6667    0.6667        18
        spam     0.2500    0.2500    0.2500         8

    accuracy                         0.5385        26
   macro avg     0.4583    0.4583    0.4583        26
weighted avg     0.5385    0.5385    0.5385        26



In [14]:

out = pd.DataFrame({
    "text": X_test,
    "true": y_true,
    "pred_model_a": y_pred_a,
    "pred_model_b": y_pred_b,
})
out.to_csv("results_task3_hf_a_predictions.csv", index=False, encoding="utf-8")
print("Saved: results_task3_hf_a_predictions.csv")
out.head()


Saved: results_task3_hf_a_predictions.csv


,text,true,pred_model_a,pred_model_b
0,[The Virtual Reward Center] Re: ** Clarificati...,not spam,spam,not spam
1,Your Account Was Accessed From a New Device We...,not spam,spam,not spam
2,You've Earned a Reward from Bard Explorers Ind...,not spam,spam,not spam
3,"Job Opportunities that pay ?600,000+ and up in...",spam,spam,spam
4,Changes to our Terms and Conditions and Privac...,not spam,not spam,not spam


## (b) Українські приклади pipeline

In [15]:

uk2en = pipeline("translation", model="Helsinki-NLP/opus-mt-uk-en", device=device_id)

ua_text = "Я працюю аналітиком SOC і розслідую фішингові інциденти."
print("UA:", ua_text)
print("EN:", uk2en(ua_text, max_length=128)[0]["translation_text"])


/home/kali/Downloads/data_anal/lab4/.venv/lib/python3.13/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


UA: Я працюю аналітиком SOC і розслідую фішингові інциденти.
EN: I'm a SOC analyst and I'm investigating fiscal incidents.


In [18]:

summ = pipeline("summarization", model="ukr-models/uk-summarizer", device=device_id)

summ.tokenizer.model_max_length = 512  

long_ua = (
    "У сучасних умовах кіберзагрози еволюціонують, а фішинг стає більш персоналізованим. "
    "Організації впроваджують MFA, сегментацію мережі, моніторинг та регулярні навчання персоналу. "
    "Важливо мати план реагування на інциденти та проводити tabletop-вправи."
)

print(
    summ(
        long_ua,
        truncation=True,
        max_new_tokens=64,
        min_new_tokens=20,
        do_sample=False,
    )[0]["summary_text"]
)

Device set to use cpu


У сучасному світі кіберзагрози стають більш персоналізованими, а фішинг - більш персоналізованим.


In [19]:

zs = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", device=device_id)

text_ua = "Терміново підтвердіть пароль за посиланням, інакше акаунт буде заблоковано."
labels = ["spam", "not spam"]

res = zs(
    text_ua,
    candidate_labels=labels,
    hypothesis_template="Цей текст належить до категорії {}.",
    multi_label=False
)
res


Device set to use cpu


{'sequence': 'Терміново підтвердіть пароль за посиланням, інакше акаунт буде заблоковано.',
 'labels': ['not spam', 'spam'],
 'scores': [0.7209580540657043, 0.27904200553894043]}